In [2]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [3]:
inter_df = pd.read_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checkpoint_1.csv')
inter_df['사용승인_일'] = inter_df['사용승인_일'].astype('Int64')

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def approved_year(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        return int(str(x)[:4])

In [8]:
inter_df['사용승인_년도'] = inter_df['사용승인_일'].apply(approved_year)

In [6]:
inv_df = inter_df[inter_df['사용승인_년도'] == 9920]
inter_df.loc[inv_df.index, '사용승인_일'] = 19920130
inv_df = inter_df[inter_df['사용승인_년도'] == 1111]
inter_df.loc[inv_df.index, '사용승인_일'] = 19111111

In [9]:
inter_df['사용승인_년도'] = inter_df['사용승인_일'].apply(approved_year)

In [10]:
inter_df['사용승인_년도'].describe()

count    889750.000000
mean       1999.807538
std          11.018245
min        1900.000000
25%        1992.000000
50%        2002.000000
75%        2008.000000
max        2020.000000
Name: 사용승인_년도, dtype: float64

In [13]:
inter_df['기준년월_최초사용승인일'] = inter_df[['지번주소', '기준년월', '사용승인_일']].groupby(['지번주소', '기준년월']).transform('min')['사용승인_일']
inter_df['기준년월_최근사용승인일'] = inter_df[['지번주소', '기준년월', '사용승인_일']].groupby(['지번주소', '기준년월']).transform('max')['사용승인_일']

In [14]:
inter_df['주소별_최초생성일자'] = inter_df[['지번주소', '생성_일자']].groupby(['지번주소']).transform('min')['생성_일자']
inter_df['주소별_최근생성일자'] = inter_df[['지번주소', '생성_일자']].groupby(['지번주소']).transform('max')['생성_일자']

In [15]:
inter_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인_년도
0,11440-21639,집합,표제부,0,177.62,105.300,59.28,393.130,385.210,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20040323,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반,20040323,20040323,20101230,20170209,2004.0
1,11590-100182663,집합,표제부,1,389.48,232.420,59.67,947.740,659.890,169.43,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080826,NaN,0.0,0,NaN,0,NaN,0,20110119,201412,서울특별시 동작구 상도동 302-28,10,일반,20080826,20080826,20110119,20190530,2008.0
2,11320-100182241,집합,표제부,0,208.00,81.085,38.98,268.165,268.165,128.93,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080721,NaN,0.0,0,NaN,0,NaN,0,20110202,201412,서울특별시 도봉구 쌍문동 380-24,4,일반,20080721,20080721,20110202,20190530,2008.0
3,11440-32174,집합,표제부,0,314.00,180.380,57.45,627.880,627.880,199.96,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,평스라브,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050901,NaN,0.0,0,NaN,0,NaN,0,20110125,201412,서울특별시 마포구 성산동 200-260,8,일반,20050901,20050901,20110125,20171205,2005.0
4,11740-100179671,집합,표제부,0,0.00,564.010,0.00,6401.190,6401.190,0.00,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20090311,NaN,0.0,0,NaN,0,NaN,0,20110122,201412,서울특별시 강동구 강일동 674-0,78,일반,20090311,20090311,20100918,20200428,2009.0


In [20]:
inter_df['주소별_기준년월_총세대수합'] = inter_df[['지번주소', '기준년월', '총세대수']].groupby(['지번주소', '기준년월']).transform('sum')['총세대수']
inter_df['주소별_기준년월_총세대수합'].describe()

count    894005.000000
mean        185.387007
std         622.680007
min           0.000000
25%           7.000000
50%           9.000000
75%          19.000000
max        6920.000000
Name: 주소별_기준년월_총세대수합, dtype: float64

In [21]:
inter_df.to_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checkpoint_2.csv', index=False)

In [22]:
total_house_number_df = inter_df[['지번주소', '관리_건축물대장_PK', '기준년월', '주소별_기준년월_총세대수합', '기준년월_최초사용승인일', '주소별_최초생성일자']].drop_duplicates().sort_values(['지번주소', '기준년월'])
print(total_house_number_df.shape)
total_house_number_df.head()

(894005, 6)


,지번주소,관리_건축물대장_PK,기준년월,주소별_기준년월_총세대수합,기준년월_최초사용승인일,주소별_최초생성일자
111184,서울특별시 강남구 개포동 1163-4,11680-100218729,201412,20,20130307,20130313
892572,서울특별시 강남구 개포동 1163-4,11680-100218729,201512,20,20130307,20130313
215664,서울특별시 강남구 개포동 1163-4,11680-100218729,201612,20,20130307,20130313
310253,서울특별시 강남구 개포동 1163-4,11680-100218729,201712,20,20130307,20130313
437662,서울특별시 강남구 개포동 1163-4,11680-100218729,201812,20,20130307,20130313


In [23]:
total_house_number_df.to_csv('./prepped_data/표제부_기준년월별_총세대수합.csv', index=False)